In [1]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data

print("Loading Data...")
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 

def add_layer(name, inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    #Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    Weights = tf.get_variable(name, shape=[in_size, out_size], initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
        mode='FAN_IN', uniform=False, seed=None, dtype=tf.float32))
    #biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    biases = tf.get_variable("b"+name, shape=[1, out_size], initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
        mode='FAN_IN', uniform=False, seed=None, dtype=tf.float32))
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

def one_hot_encode(dense_data, depth):
    length = len(dense_data)
    print("length:",length)
    b = np.zeros((length, depth))
    b[np.arange(length), dense_data] = 1
    return np.array(b)

depth = 5
y_train1 = one_hot_encode(y_train1, depth)

#y_train1 = y_train1[:, np.newaxis]
print(y_train1.shape)
y_valid1 = y_valid1[:, np.newaxis]
y_test1 = y_test1[:, np.newaxis]
  
_, dim = X_train1.shape
print(dim)
print(y_train1)
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, dim])
ys = tf.placeholder(tf.float32, [None, depth])

# add hidden layer
l1 = add_layer("1",xs, dim, 128, activation_function=tf.nn.elu)
l2 = add_layer("2", l1, 128, 128, activation_function=tf.nn.elu)
l3 = add_layer("3", l2, 128, 128, activation_function=tf.nn.elu)
l4 = add_layer("4", l3, 128, 128, activation_function=tf.nn.elu)
l5 = add_layer("5", l4, 128, 128, activation_function=tf.nn.elu)

#W = tf.Variable(tf.zeros([784, 1]))
#W = tf.get_variable("W", shape=[784, 1], initializer=tf.contrib.layers.xavier_initializer(
#    shape=[784, 1],variance=4. / (100. + 40.), factor=2.0))

y = tf.nn.softmax(add_layer("output", l5, 128, 5, activation_function=None))
print(y.shape)

cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = ys))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()
for i in range(30):
    if(i%20==0):
        print("Steps:",i)
    #batch_xs, batch_ys = X_train1.next_batch(100)
    #sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    sess.run(train_step, feed_dict={xs: X_train1, ys: y_train1})

print(sess.run(y[100], feed_dict={xs: X_train1}))
correct_prediction = tf.equal(y, ys)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={xs: X_train1, ys: y_train1}))

Loading Data...
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
length: 28038
(28038, 5)
784
[[ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.]
 ..., 
 [ 0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]]
(?, 5)
Steps: 0
Steps: 20
[ 0.  0.  1.  0.  0.]
0.678037
